In [1]:
import time
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from collections import defaultdict
from itertools import combinations
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network

In [2]:
opportunities = pd.read_pickle('../Data/2022-10-24.pkl')

In [3]:
index = 'URL'
assert opportunities[index].value_counts().max() == 1

In [4]:
discipline = opportunities[[index, 'DisciplineNames']].explode('DisciplineNames')

In [5]:
subject_name = opportunities[[index, 'SubjectNames']].explode('SubjectNames')

In [6]:
discipline

,URL,DisciplineNames
0,https://www.findaphd.com/phds/project/swbio-dt...,Biological Sciences
0,https://www.findaphd.com/phds/project/swbio-dt...,Mathematics
0,https://www.findaphd.com/phds/project/swbio-dt...,Veterinary Sciences
1,https://www.findaphd.com/phds/project/carbon-s...,Biological Sciences
1,https://www.findaphd.com/phds/project/carbon-s...,Environmental Sciences
...,...,...
5519,https://www.findaphd.com/phds/programme/the-in...,Sport & Exercise Science
5520,https://www.findaphd.com/phds/program/phds-at-...,Literature & Culture
5520,https://www.findaphd.com/phds/program/phds-at-...,Computer Science
5520,https://www.findaphd.com/phds/program/phds-at-...,Languages


In [7]:
opportunities.sample(10)

,Title,Description,URL,Deadline,SearchDate,DisciplineIds,DisciplineNames,SubjectIds,SubjectNames,LocationCountryName,LocationCityName,ProgrammeTypes,IId,InstitutionName,DId,DepartmentName,FundingTypes
402,Enhancing Performance of Structural Materials ...,"In this project, you will develop a predictive...",https://www.findaphd.com/phds/project/enhancin...,NaT,2022-10-24,"(12, 24)","(Engineering, Materials Science)","(231, 233, 241, 249, 250, 252, 261, 263, 371)","(Aerospace Engineering, Automotive Engineering...",United Kingdom,Sheffield,PhD Research Project,348,University of Sheffield,1638,Department of Materials Science and Engineering,"(SF, UK)"
574,Clinically-motivated and physics-informed deep...,Contouring tumour volumes is an important earl...,https://www.findaphd.com/phds/project/clinical...,NaT,2022-10-24,"(26, 4)","(Biological Sciences, Medicine)","(121, 122, 123, 150, 407, 411)","(Bioinformatics, Biophysics, Biotechnology, Ot...",United Kingdom,Oxford,PhD Research Project,339,University of Oxford,1974,Department of Oncology,"(EU, NE, SF, UK)"
5432,Decarbonisation of the Food System,As the world transitions to a low carbon econo...,https://www.findaphd.com/phds/project/decarbon...,NaT,2022-10-24,"(1, 12, 13, 15, 17, 25, 26, 3, 4)","( Building & Planning, Agriculture, Architectu...","(101, 113, 115, 116, 130, 244, 245, 265, 281, ...","(Agricultural Sciences, Climate Science, Data ...",United Kingdom,Reading,PhD Research Project,345,University of Reading,3959,School of the Built Environment,SF
2383,Mitochondrial transport proteins involved in t...,Complex I (NADH:ubiquinone oxidoreductase) is ...,https://www.findaphd.com/phds/project/mitochon...,NaT,2022-10-24,4,Biological Sciences,"(119, 122, 125, 133, 139, 146)","(Biochemistry, Biophysics, Cell Biology, Genet...",United Kingdom,Cambridge,PhD Research Project,290,University of Cambridge,4420,MRC Mitochondrial Biology Unit,"(EU, NE, SF, UK)"
575,Using Comparative Genomics to Develop ‘digital...,Project Description. Our rapidly changing worl...,https://www.findaphd.com/phds/project/using-co...,NaT,2022-10-24,"(13, 4, 8)","(Biological Sciences, Computer Science, Enviro...","(121, 128, 130, 131, 134, 186, 191, 197, 269, ...","(Artificial Intelligence, Bioinformatics, Data...",United Kingdom,Cardiff,PhD Research Project,33,Cardiff University,259,Cardiff School of Biosciences,"(EU, NE, SF, UK)"
1606,Investigating the rescue of cystic fibrosis-ca...,A 3-year PhD studentship funded by the Cystic ...,https://www.findaphd.com/phds/project/investig...,NaT,2022-10-24,"(25, 26, 4)","(Biological Sciences, Mathematics, Medicine)","(119, 122, 135, 140, 146, 378, 385, 386, 402, ...","(Biochemistry, Biophysics, Data Analysis, Huma...",United Kingdom,London,PhD Research Project,274,University College London,7103,UCL Division of Biosciences,"(SF, UK)"
4500,PhD Education,OVERVIEW. You'll be part of a dynamic doctoral...,https://www.findaphd.com/phds/programme/phd-ed...,NaT,2022-10-24,11,Education,229,Other,United Kingdom,Belfast,Social Sciences Research Programme,195,Queen’s University Belfast,,,SF
5094,Development of latent fingermarks on challengi...,Research Group. Forensic and Investigative Sci...,https://www.findaphd.com/phds/project/developm...,NaT,2022-10-24,16,Forensic and Archaeological Sciences,283,Forensic Science,United Kingdom,Cambridge,PhD Research Project,7,Anglia Ruskin University ARU,3815,School of Life Sciences,SF
5276,SFI Centre for Research Training in Digitally ...,The SFI Centre for Research Training in Digita...,https://www.findaphd.com/phds/program/sfi-cent...,NaT,2022-10-24,"(12, 8)","(Computer Science, Engineering)","(199, 264)","(Other, Other)",Ireland,Galway,Ireland PhD Programme,172,"( Galway, National University of Ireland)",,,"(EU, NE, SF, UK)"
4387,Humanities in Lampeter Research,"Established in 1822, Lampeter has a distinguis...",https://www.findaphd.com/phds/programme/humani...,NaT,2022-10-24,"(17, 19, 21, 28, 34)","( Literature & Culture, Geography, Histo

In [15]:
subject_name

,URL,SubjectNames
0,https://www.findaphd.com/phds/project/swbio-dt...,Bacteriology
0,https://www.findaphd.com/phds/project/swbio-dt...,Biodiversity
0,https://www.findaphd.com/phds/project/swbio-dt...,Bioinformatics
0,https://www.findaphd.com/phds/project/swbio-dt...,Data Analysis
0,https://www.findaphd.com/phds/project/swbio-dt...,Environmental Biology
...,...,...
5519,https://www.findaphd.com/phds/programme/the-in...,Other
5519,https://www.findaphd.com/phds/programme/the-in...,Physiology
5520,https://www.findaphd.com/phds/program/phds-at-...,Other
5520,https://www.findaphd.com/phds/program/phds-at-...,Other


In [8]:
counts = defaultdict(int)
for index, row in opportunities.iterrows():
    names = []
    for col in ['SubjectNames', 'DisciplineNames']:
        if isinstance(row[col], tuple):
            names.extend(list(row[col]))
        else:
            names.append(row[col])
    names = [i for i in names if i not in ['', 'Other']]
        
    for pair in combinations(names, 2):
        pair = tuple(sorted(pair))  
        counts[pair] += 1
        
counts = pd.Series(counts).reset_index()
counts = counts.loc[counts[0] > 20]
maxCount = counts[0].max()
counts = counts.apply(tuple, axis=1).to_list()

In [9]:
G = nx.Graph()
G.add_weighted_edges_from(counts)

In [10]:
topic = 'Evolution'
subGraph = nx.ego_graph(G, topic, radius=1)
for u, v, d in subGraph.edges(data=True):
    d['weight'] = (d['weight'] / maxCount) * 3
nt = Network('500px', '500px')
nt.show_buttons(filter_=['physics']) 
nt.from_nx(subGraph)
nt.show(f'nx-{topic}.html')

In [11]:
counts

[('Bacteriology', 'Bioinformatics', 24),
 ('Bacteriology', 'Microbiology', 47),
 ('Bacteriology', 'Biological Sciences', 64),
 ('Biodiversity', 'Bioinformatics', 25),
 ('Biodiversity', 'Environmental Biology', 35),
 ('Biodiversity', 'Evolution', 25),
 ('Biodiversity', 'Genomics', 25),
 ('Biodiversity', 'Biological Sciences', 79),
 ('Bioinformatics', 'Data Analysis', 93),
 ('Bioinformatics', 'Environmental Biology', 21),
 ('Bioinformatics', 'Evolution', 96),
 ('Bioinformatics', 'Genomics', 96),
 ('Bioinformatics', 'Microbiology', 97),
 ('Bioinformatics', 'Biological Sciences', 525),
 ('Bioinformatics', 'Mathematics', 120),
 ('Biological Sciences', 'Data Analysis', 169),
 ('Data Analysis', 'Mathematics', 385),
 ('Environmental Biology', 'Evolution', 21),
 ('Environmental Biology', 'Microbiology', 31),
 ('Biological Sciences', 'Environmental Biology', 143),
 ('Evolution', 'Genomics', 48),
 ('Evolution', 'Microbiology', 56),
 ('Biological Sciences', 'Evolution', 206),
 ('Evolution', 'Mathe

In [12]:
nt = Network('500px', '500px')
nt.from_nx(G)
nt.show('nx.html')

In [13]:
len(subGraph.edges())

143

In [14]:
for node in subGraph.edges():
    print(node)

('Plant Biology', 'Ecology')
('Plant Biology', 'Biological Sciences')
('Plant Biology', 'Biochemistry')
('Plant Biology', 'Microbiology')
('Plant Biology', 'Environmental Sciences')
('Plant Biology', 'Agricultural Sciences')
('Plant Biology', 'Agriculture')
('Plant Biology', 'Bioinformatics')
('Plant Biology', 'Evolution')
('Plant Biology', 'Genetics')
('Plant Biology', 'Molecular Biology')
('Plant Biology', 'Cell Biology')
('Cell Biology', 'Molecular Biology')
('Cell Biology', 'Biological Sciences')
('Cell Biology', 'Bioinformatics')
('Cell Biology', 'Genetics')
('Cell Biology', 'Biochemistry')
('Cell Biology', 'Medicine')
('Cell Biology', 'Evolution')
('Cell Biology', 'Genomics')
('Cell Biology', 'Microbiology')
('Cell Biology', 'Mathematics')
('Cell Biology', 'Computer Science')
('Biochemistry', 'Molecular Biology')
('Biochemistry', 'Biological Sciences')
('Biochemistry', 'Medicine')
('Biochemistry', 'Microbiology')
('Biochemistry', 'Bioinformatics')
('Biochemistry', 'Evolution')
('

In [16]:
subject_name.to_csv('../Data/FindaPhD_subject_name.csv')

In [18]:
discipline.to_csv('../Data/FindaPhD_discipline_name.csv')